In [40]:
import requests
import urllib.parse
import json

def search_item(keyword):
    # 關鍵字做 URL 編碼
    kw = urllib.parse.quote(keyword)

    api_url = f"https://ecshweb.pchome.com.tw/search/v3.3/all/results?q={kw}&page=1&sort=rnk/dc"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    r = requests.get(api_url, headers=headers, timeout=10)
    r.raise_for_status()
    data = r.json()

    prods = data.get("prods") or []
    if not prods:
        return "無此商品"

    results = []

    for item in prods[:25]:
        name = item.get("name", "無資料")
        price = item.get("price", "無資料")

        product_id = item.get("Id", "")
        url = f"https://24h.pchome.com.tw/prod/{product_id}" if product_id else ""

        # 圖片路徑：用 picB / picS，前綴改成 cs-a.ecimg.tw
        pic_path = item.get("picB") or item.get("picS") or ""
        img = f"https://cs-a.ecimg.tw{pic_path}" if pic_path else ""

        results.append({
            "name": name,
            "price": price,
            "url": url,
            "img": img
        })

    # 存成 JSON 檔
    filename = f"{keyword}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    return results


# 測試
data = search_item("牛奶")
if isinstance(data, str):
    print(data)
else:
    for i, item in enumerate(data, 1):
        print(f"第 {i} 筆：")
        print(f"  名稱：{item['name']}")
        print(f"  價格：{item['price']}")
        print(f"  商品：{item['url']}")
        print(f"  圖片：{item['img']}")
        print("-" * 40)


第 1 筆：
  名稱：成份無調整保久乳-全脂牛乳200ml(24入/箱)x2
  價格：899
  商品：https://24h.pchome.com.tw/prod/DBAB08-1900I8G9J
  圖片：https://cs-a.ecimg.tw/items/DBAB081900I8G9J/000001_1760492594.jpg
----------------------------------------
第 2 筆：
  名稱：【南紡購物中心】 紐西蘭AnchorSGS認證1公升100%純牛奶保久乳(1Lx3瓶組合) 
  價格：459
  商品：https://24h.pchome.com.tw/prod/DBDA0G-A900AZ32U
  圖片：https://cs-a.ecimg.tw/items/DBDA0GA900AZ32U/000001_1656407706.jpg
----------------------------------------
第 3 筆：
  名稱：高鈣牛乳-無加糖200ml 24入
  價格：312
  商品：https://24h.pchome.com.tw/prod/DBAB7J-A90090XUI
  圖片：https://cs-a.ecimg.tw/items/DBAB7JA90090XUI/000001_1661614703.jpg
----------------------------------------
第 4 筆：
  名稱：保久乳-低脂高鈣牛乳200ml(24入)
  價格：269
  商品：https://24h.pchome.com.tw/prod/DBAB08-A49138250
  圖片：https://cs-a.ecimg.tw/items/DBAB08A49138250/000001_1545812747.jpg
----------------------------------------
第 5 筆：
  名稱：W PROTEIN JUNIOR成長期乳清蛋白飲-巧克力牛奶(35g*10包/盒)
  價格：320
  商品：https://24h.pchome.com.tw/prod/DFAX00-A900HB4NO
  圖片：https://cs-a.ecimg.tw

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json   

def search_pchome(keyword: str, max_items: int = 10):
    """
    Use Selenium to search PChome.
    Returns:
      - If found: list[dict], each {name, price, img, link}
      - If nothing: "無此商品"
    """
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--incognito")
    options.add_argument("--disable-popup-blocking")

    driver = webdriver.Chrome(options=options)
    CARD = ".c-prodInfoV2.c-prodInfoV2--gridCard"

    try:
        # Go to search result page
        driver.get(f"https://24h.pchome.com.tw/search/?q={keyword}")
        time.sleep(3)

        # First batch of product cards
        items = driver.find_elements(By.CSS_SELECTOR, CARD)
        if not items:
            return "無此商品"

        last = items[-1]

        # Scroll down until no new items or enough items
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            new_items = driver.find_elements(By.CSS_SELECTOR, CARD)
            if not new_items or new_items[-1] == last or len(new_items) >= max_items * 3:
                items = new_items
                break

            items = new_items
            last = items[-1]

        results = []

        for card in items:
            # Scroll this card into view to trigger lazy-load image
            driver.execute_script("arguments[0].scrollIntoView();", card)
            time.sleep(0.2)

            # Name
            name = card.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__title").text
            if keyword not in name:
                continue

            # Price
            price = card.find_element(
                By.CSS_SELECTOR,
                ".c-prodInfoV2__priceValue.c-prodInfoV2__priceValue--m"
            ).text

            # Image (handle mobile_loading.svg lazy-load)
            img_el = card.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__img img")
            img = img_el.get_attribute("src") or ""
            if "mobile_loading.svg" in img:
                for attr in ("data-src", "data-srcset", "srcset"):
                    v = img_el.get_attribute(attr)
                    if v:
                        img = v.split(",")[0].strip().split(" ")[0]
                        break
            if not img:
                img = "無圖片"

            # Product link
            link = card.find_element(
                By.CSS_SELECTOR,
                "a.c-prodInfoV2__link.gtmClickV2"
            ).get_attribute("href")

            results.append({
                "name": name,
                "price": price,
                "img": img,
                "link": link
            })

            if len(results) >= max_items:
                break

        # ---- ★ Save to JSON here ★ ----
        if results:
            filename = f"{keyword}.json"   # e.g. "衛生紙.json"
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            return results
        else:
            return "無此商品"

    finally:
        driver.quit()


if __name__ == "__main__":
    kw = "找不到的輸出結果"
    data = search_pchome(kw, max_items=50)

    if isinstance(data, str):
        # e.g. "無此商品"
        print(f"關鍵字「{kw}」搜尋結果：{data}")
    else:
        print(f"關鍵字「{kw}」找到 {len(data)} 筆商品：\n")
        for i, item in enumerate(data, start=1):
            print(f"第 {i} 筆")
            print(f"  名稱：{item['name']}")
            print(f"  價格：{item['price']}")
            print(f"  連結：{item['link']}")
            print(f"  圖片：{item['img']}")
            print("-" * 50)


關鍵字「找不到的輸出結果」搜尋結果：無此商品
